In [145]:
import os, sys, json
import numpy as np
import pandas as pd
import re
import ast
from pprint import pprint



def move_working_dir_to_repo_root(repo_name="orgsync"):
    """
    Move the current working directory to the root of the repository.
    """
    current_dir = os.getcwd()
    while os.path.basename(current_dir).lower() != repo_name:
        current_dir = os.path.dirname(current_dir)
    os.chdir(current_dir)
    print("Current working directory: ", os.getcwd())

move_working_dir_to_repo_root(repo_name="orgsync")


data_dir = "data/geolocations/orgs_names_postcodes.csv"
df = pd.read_csv(data_dir)
# remove nans
df = df.dropna(subset=['postcode'])

Current working directory:  c:\Users\dec2g\GitHub\OrgSync


In [146]:
df.head(50)

,dataset,id,name,short_name,postcode,city
0,cordis,323260,nuclear decommissioning authority - nda,nda,ca24 3hu,moor row
1,cordis,219438,south west tourism limited,south west tourism,ex2 5wt,exeter
2,cordis,219438,welsh government,welsh government,cf10 3nq,cardiff
3,cordis,226103,terrasalus limited,terrasalus,le15 9el,oakham
4,cordis,282558,ol pharma partners ltd,pharmivation,sg4 7dp,weston herts
5,cordis,601714,forsite diagnostics ltd,forsite diagnostics,yo41 1lz,york
6,cordis,241839,st george's hospital medical school,NaN,sw17 0re,london
7,cordis,201720,genetic alliance uk ltd,gauk,n1 3qp,london
8,cordis,601852,fetal medicine foundation,fetal medicine found,w1g 6bg,london
9,cordis,241873,centre of the cell,centre of the cell,e1 2ab,london


In [147]:
# convert postcode to lowercase
df['postcode'] = df['postcode'].str.lower()

# remove any leading or trailing whitespace
df['postcode'] = df['postcode'].str.strip()

# remove non-alphanumeric characters
df['postcode'] = df['postcode'].str.replace(r'\W+', '')

df = df[df["dataset"] == "gtr"].copy()

df.head()

,dataset,id,name,short_name,postcode,city
37245,gtr,5331B126-3AB4-4412-B56D-00E8F2796556,newcastle city council,NaN,ne1 8qh,NaN
37246,gtr,53331120-0290-49FA-A513-0286A214AF7A,valerann uk limited,NaN,mk14 6gd,NaN
37248,gtr,77908BF8-1B2D-4D26-9119-155100E8B9C5,mindray,NaN,pe29 6fn,NaN
37249,gtr,7794C645-9CC7-4913-A8DC-103AE0EFDD4B,democracy international,NaN,20814,NaN
37251,gtr,53474E16-DA35-4007-9F07-1B6C0F32BBF6,licx uk limited,NaN,hp7 0ut,NaN


# NOTE - I'm just looking at Gtr at this point but we can remove `df = df[df["dataset"] == "GtR"]` to look at all postcodes

# Note - Some postcodes have an o instead of a 0 and we aren't catching these. We could add them back in later

In [148]:
from uk_postcodes_parsing import ukpostcode

# corpus = "ca24 3hu"
# parsed = ukpostcode.parse_from_corpus(corpus)[0]
# print(parsed)
# print(type(parsed))
# print(parsed.__dict__)
# (is_in_ons_postcode_directory=True, fix_distance=0, original='ca24 3hu', postcode='CA24 3HU', incode='3HU', outcode='CA24', area='CA', district='CA24', sub_district=None, sector='CA24 3', unit='HU')

def parse(postcode):
    return ukpostcode.parse_from_corpus(postcode)

df["parsed_postcode"] = df["postcode"].apply(lambda x: parse(x)[0].__dict__ if len(parse(x))>0 else None) 

df.head(50)

,dataset,id,name,short_name,postcode,city,parsed_postcode
37245,gtr,5331B126-3AB4-4412-B56D-00E8F2796556,newcastle city council,NaN,ne1 8qh,NaN,"{'original': 'ne1 8qh', 'postcode': 'NE1 8QH',..."
37246,gtr,53331120-0290-49FA-A513-0286A214AF7A,valerann uk limited,NaN,mk14 6gd,NaN,"{'original': 'mk14 6gd', 'postcode': 'MK14 6GD..."
37248,gtr,77908BF8-1B2D-4D26-9119-155100E8B9C5,mindray,NaN,pe29 6fn,NaN,"{'original': 'pe29 6fn', 'postcode': 'PE29 6FN..."
37249,gtr,7794C645-9CC7-4913-A8DC-103AE0EFDD4B,democracy international,NaN,20814,NaN,None
37251,gtr,53474E16-DA35-4007-9F07-1B6C0F32BBF6,licx uk limited,NaN,hp7 0ut,NaN,"{'original': 'hp7 0ut', 'postcode': 'HP7 0UT',..."
37252,gtr,5370084A-BDBF-4A7B-8E02-09B7DE95D01B,panitek power limited,NaN,ec1v 2nx,NaN,"{'original': 'ec1v 2nx', 'postcode': 'EC1V 2NX..."
37253,gtr,53700B68-0910-469D-A0CB-18A3316DF3F2,agency for healthcare research and quality,NaN,unspecified,NaN,None
37254,gtr,537D521C-3D73-458B-A054-196982ADD766,control developments (uk) ltd,NaN,ox29 4pu,NaN,"{'original': 'ox29 4pu', 'postcode': 'OX29 4PU..."
37255,gtr,538761FB-F8B6-4799-9ADA-02F4EF15E8B3,university of santo tomas,NaN,1008,NaN,None
37256,gtr,538EFD3B-D8AF-4B06-BB0E-122E58E65FEF,aberdeen harbour board,NaN,ab11 5ss,NaN,"{'original': 'ab11 5ss', 'postcode': 'AB11 5SS..."


In [149]:
# find all entries with postcode = "CB2 1TN"
df_cb2 = df[df['postcode'] == "cb2 1tn"]
df_cb2.head(50)

,dataset,id,name,short_name,postcode,city,parsed_postcode
83181,gtr,EDAC9A61-A5DE-4A96-A89F-C793E0EC8681,cambridge enterprise limited,NaN,cb2 1tn,NaN,"{'original': 'cb2 1tn', 'postcode': 'CB2 1TN',..."
87740,gtr,8E38A887-3CD8-49FD-87E5-A1844E81B9CF,university of cambridge,NaN,cb2 1tn,NaN,"{'original': 'cb2 1tn', 'postcode': 'CB2 1TN',..."


In [150]:
# remove None values
df = df.dropna(subset=['parsed_postcode'])
df.head(50)

,dataset,id,name,short_name,postcode,city,parsed_postcode
37245,gtr,5331B126-3AB4-4412-B56D-00E8F2796556,newcastle city council,NaN,ne1 8qh,NaN,"{'original': 'ne1 8qh', 'postcode': 'NE1 8QH',..."
37246,gtr,53331120-0290-49FA-A513-0286A214AF7A,valerann uk limited,NaN,mk14 6gd,NaN,"{'original': 'mk14 6gd', 'postcode': 'MK14 6GD..."
37248,gtr,77908BF8-1B2D-4D26-9119-155100E8B9C5,mindray,NaN,pe29 6fn,NaN,"{'original': 'pe29 6fn', 'postcode': 'PE29 6FN..."
37251,gtr,53474E16-DA35-4007-9F07-1B6C0F32BBF6,licx uk limited,NaN,hp7 0ut,NaN,"{'original': 'hp7 0ut', 'postcode': 'HP7 0UT',..."
37252,gtr,5370084A-BDBF-4A7B-8E02-09B7DE95D01B,panitek power limited,NaN,ec1v 2nx,NaN,"{'original': 'ec1v 2nx', 'postcode': 'EC1V 2NX..."
37254,gtr,537D521C-3D73-458B-A054-196982ADD766,control developments (uk) ltd,NaN,ox29 4pu,NaN,"{'original': 'ox29 4pu', 'postcode': 'OX29 4PU..."
37256,gtr,538EFD3B-D8AF-4B06-BB0E-122E58E65FEF,aberdeen harbour board,NaN,ab11 5ss,NaN,"{'original': 'ab11 5ss', 'postcode': 'AB11 5SS..."
37258,gtr,5396B4CA-422A-4ACB-88AC-1BF02F1D28D3,ak industries limited,NaN,hr2 6jq,NaN,"{'original': 'hr2 6jq', 'postcode': 'HR2 6JQ',..."
37260,gtr,5398E19C-4A38-4777-9DB1-0224648914E0,kool zone limited,NaN,rg9 1nx,NaN,"{'original': 'rg9 1nx', 'postcode': 'RG9 1NX',..."
37262,gtr,53AB545D-9946-4C47-B670-07AA591E9CFF,3d merchandising limited,NaN,bb1 1hw,NaN,"{'original': 'bb1 1hw', 'postcode': 'BB1 1HW',..."


In [151]:
df.tail()

,dataset,id,name,short_name,postcode,city,parsed_postcode
106305,gtr,01F2924C-FFB4-481B-B8F0-31234D33F0FA,university of exeter,NaN,ex4 4sb,NaN,"{'original': 'ex4 4sb', 'postcode': 'EX4 4SB',..."
106306,gtr,0433C2E4-BD74-4E19-A10F-04877DD9411B,spearhead marketing limited,NaN,cb8 8jp,NaN,"{'original': 'cb8 8jp', 'postcode': 'CB8 8JP',..."
106307,gtr,0648C484-FB54-4EE8-8AA5-0CCC73A1F54C,bgr training limited,NaN,g41 3qs,NaN,"{'original': 'g41 3qs', 'postcode': 'G41 3QS',..."
106310,gtr,0D4FDCE2-32C8-4FC1-A582-331BEE2B5C1F,disyn biotc,NaN,wa5 3pz,NaN,"{'original': 'wa5 3pz', 'postcode': 'WA5 3PZ',..."
106311,gtr,E15F7794-DE6A-4BF3-814D-075FDAE2E332,x-io technologies limited,NaN,bs4 2js,NaN,"{'original': 'bs4 2js', 'postcode': 'BS4 2JS',..."


In [152]:
# get type of parsed_postcode
print(type(df.iloc[0]["parsed_postcode"]))

<class 'dict'>


In [153]:
df.tail(10)

,dataset,id,name,short_name,postcode,city,parsed_postcode
106297,gtr,DC85C5E8-F95E-48A9-B3E8-23110BF82DF9,ticker limited,NaN,gu8 6bq,NaN,"{'original': 'gu8 6bq', 'postcode': 'GU8 6BQ',..."
106298,gtr,DDBC0327-903E-4CC0-970F-231601349D9E,ekaterra r&d,NaN,mk44 1lq,NaN,"{'original': 'mk44 1lq', 'postcode': 'MK44 1LQ..."
106300,gtr,DE0B2CDD-AF6A-4E4B-83C2-0A70968D934B,hdr uk,NaN,nw1 2be,NaN,"{'original': 'nw1 2be', 'postcode': 'NW1 2BE',..."
106301,gtr,DE2A3BD5-4503-420D-AC8A-1C14E0AC401F,sartorius stedim lab limited,NaN,gl10 3ut,NaN,"{'original': 'gl10 3ut', 'postcode': 'GL10 3UT..."
106303,gtr,E17A6DDD-8D5F-4E58-8AB0-2628D5266F1E,recon waste management ltd,NaN,bt62 1ux,NaN,"{'original': 'bt62 1ux', 'postcode': 'BT62 1UX..."
106305,gtr,01F2924C-FFB4-481B-B8F0-31234D33F0FA,university of exeter,NaN,ex4 4sb,NaN,"{'original': 'ex4 4sb', 'postcode': 'EX4 4SB',..."
106306,gtr,0433C2E4-BD74-4E19-A10F-04877DD9411B,spearhead marketing limited,NaN,cb8 8jp,NaN,"{'original': 'cb8 8jp', 'postcode': 'CB8 8JP',..."
106307,gtr,0648C484-FB54-4EE8-8AA5-0CCC73A1F54C,bgr training limited,NaN,g41 3qs,NaN,"{'original': 'g41 3qs', 'postcode': 'G41 3QS',..."
106310,gtr,0D4FDCE2-32C8-4FC1-A582-331BEE2B5C1F,disyn biotc,NaN,wa5 3pz,NaN,"{'original': 'wa5 3pz', 'postcode': 'WA5 3PZ',..."
106311,gtr,E15F7794-DE6A-4BF3-814D-075FDAE2E332,x-io technologies limited,NaN,bs4 2js,NaN,"{'original': 'bs4 2js', 'postcode': 'BS4 2JS',..."


In [154]:
# rename postcode column to original_postcode
df = df.rename(columns={"postcode": "original_postcode"})
df_normalised = pd.json_normalize(df['parsed_postcode'])
df_combined = pd.concat([df.reset_index(drop=True), df_normalised.reset_index(drop=True)], axis=1)


# save
df_combined.to_csv("data/geolocations/orgs_names_postcodes_cleaned.csv", index=False)

In [155]:
df_combined.tail(50)

,dataset,id,name,short_name,original_postcode,city,parsed_postcode,original,postcode,incode,outcode,area,district,sub_district,sector,unit,fix_distance,is_in_ons_postcode_directory
35921,gtr,A623CD38-0CB2-4672-B4CB-70D35D3B0083,disruptieve ltd,NaN,ec2a 4ne,NaN,"{'original': 'ec2a 4ne', 'postcode': 'EC2A 4NE...",ec2a 4ne,EC2A 4NE,4NE,EC2A,EC,EC2,EC2A,EC2A 4,NE,0,True
35922,gtr,A8E654D3-E2CB-4730-8755-639FE48000FD,legrand electric limited,NaN,b19 2lf,NaN,"{'original': 'b19 2lf', 'postcode': 'B19 2LF',...",b19 2lf,B19 2LF,2LF,B19,B,B19,None,B19 2,LF,0,True
35923,gtr,ABA2B172-B299-4585-B5D8-601DDE5F620E,quic (uk) limited,NaN,ll13 7gw,NaN,"{'original': 'll13 7gw', 'postcode': 'LL13 7GW...",ll13 7gw,LL13 7GW,7GW,LL13,LL,LL13,None,LL13 7,GW,0,True
35924,gtr,ABE0BBA0-6595-4B2E-894B-563D5F8D9F93,grow-wellbeing cic,NaN,ch41 6nd,NaN,"{'original': 'ch41 6nd', 'postcode': 'CH41 6ND...",ch41 6nd,CH41 6ND,6ND,CH41,CH,CH41,None,CH41 6,ND,0,True
35925,gtr,ACDFD9C2-230E-4063-B9A7-1660AC7ECB99,sector health ltd,NaN,bn1 1ad,NaN,"{'original': 'bn1 1ad', 'postcode': 'BN1 1AD',...",bn1 1ad,BN1 1AD,1AD,BN1,BN,BN1,None,BN1 1,AD,0,True
35926,gtr,ACE761D4-BBA1-45BF-A495-20A7D4273204,parallel wireless uk limited,NaN,gu47 0qa,NaN,"{'original': 'gu47 0qa', 'postcode': 'GU47 0QA...",gu47 0qa,GU47 0QA,0QA,GU47,GU,GU47,None,GU47 0,QA,0,True
35927,gtr,B4915CAB-7E45-44A7-9E07-6D48D792D88E,diamond light source limited,NaN,ox11 0de,NaN,"{'original': 'ox11 0de', 'postcode': 'OX11 0DE...",ox11 0de,OX11 0DE,0DE,OX11,OX,OX11,None,OX11 0,DE,0,True
35928,gtr,B4DE2E42-A68E-4D90-B9D5-3FBDF00D0447,censis,NaN,g1 1rd,NaN,"{'original': 'g1 1rd', 'postcode': 'G1 1RD', '...",g1 1rd,G1 1RD,1RD,G1,G,G1,None,G1 1,RD,0,True
35929,gtr,F8BF467D-E283-482B-A3A3-51B1B23F8C2D,cassels farm limited,NaN,ba12 0qq,NaN,"{'original': 'ba12 0qq', 'postcode': 'BA12 0QQ...",ba12 0qq,BA12 0QQ,0QQ,BA12,BA,BA12,None,BA12 0,QQ,0,True
35930,gtr,FE006F58-88C3-40CF-A595-4FCDB3D8A21F,planning democracy,NaN,ky11 3lg,NaN,"{'original': 'ky11 3lg', 'postcode': 'KY11 3LG...",ky11 3lg,KY11 3LG,3LG,KY11,KY,KY11,None,KY11 3,LG,0,True


In [156]:
# find all entries with postcode = "CB2 1TN"
df_cb2 = df_combined[df_combined['postcode'] == "CB2 1TN"]
df_cb2.head()

,dataset,id,name,short_name,original_postcode,city,parsed_postcode,original,postcode,incode,outcode,area,district,sub_district,sector,unit,fix_distance,is_in_ons_postcode_directory
23796,gtr,EDAC9A61-A5DE-4A96-A89F-C793E0EC8681,cambridge enterprise limited,NaN,cb2 1tn,NaN,"{'original': 'cb2 1tn', 'postcode': 'CB2 1TN',...",cb2 1tn,CB2 1TN,1TN,CB2,CB,CB2,None,CB2 1,TN,0,True
26163,gtr,8E38A887-3CD8-49FD-87E5-A1844E81B9CF,university of cambridge,NaN,cb2 1tn,NaN,"{'original': 'cb2 1tn', 'postcode': 'CB2 1TN',...",cb2 1tn,CB2 1TN,1TN,CB2,CB,CB2,None,CB2 1,TN,0,True


# We can now look at the organisations with postcodes that specify sub_distict and those that don't.
Those that don't are likely to be rough approximations of the location of the organisation. We can also match on incode, outcode, area, district and subdistrict. 

We can also see in "is_in_ons_postcode_directory" whether a postcode is no longer in use (will be set to False). Could indicate an org has moved. 

Now we can try matching...
1. Get df_id_postcode = df["id", "postcode"]

Get list of ids for each postcode

In [157]:
df_id_pc = df_combined[["id", "postcode"]].copy()

df_grouped = df_id_pc.groupby("postcode")["id"].apply(list).reset_index(name="org_ids")
df_grouped.head(50)

df_grouped.to_csv("data/geolocations/orgs_names_postcodes_grouped.csv", index=False)


remove any postcodes with less than 2 ids


In [158]:
# remove any postcodes with less than 2 ids
df_grouped = df_grouped[df_grouped["org_ids"].apply(lambda x: len(x) > 1)]


In [159]:
df_grouped.head()

,postcode,org_ids
3,AB10 1AB,"[7ED73A50-A205-49E0-A635-2CB09B1555E3, 4BD11E7..."
8,AB10 1FR,"[2C2EF413-6B81-4F48-BE30-58F0BF3291F0, D2CEE12..."
9,AB10 1FW,"[196C380E-218E-4E8B-89DA-1B84D05AD536, C1FBF68..."
10,AB10 1HA,"[B6392179-08BD-4FB2-9C7F-87140CD20E11, 90D8464..."
19,AB10 1UP,"[2C81420E-9DC4-4EF1-A779-8DB811235D40, 18C433C..."


In [160]:
# save df_grouped
df_grouped.to_csv("data/geolocations/orgs_names_postcodes_grouped_unique_only.csv", index=False)

In [161]:
# what postcode has the most ids and how many does it have?
max_ids = df_grouped["org_ids"].apply(len).max()
max_ids_postcode = df_grouped[df_grouped["org_ids"].apply(len) == max_ids]


In [162]:
print(max_ids)
print(max_ids_postcode)
print(len(max_ids_postcode))

168
       postcode                                            org_ids
22058  WC2H 9JQ  [010D8FA8-E2B1-4BB1-9FAE-354D039F7DF4, A08CFD3...
1


# One cambridge postcode has like 1000 ids... not sure why

Next step is to look within those groups and see if there are multiple org names. 

Similarly, groupby org names, and look for different postcodes. 